![Codenation](https://forum.codenation.com.br/uploads/default/original/2X/2/2d2d2a9469f0171e7df2c4ee97f70c555e431e76.png)

__Autor__: Kazuki Yokoyama (kazuki.yokoyama@ufrgs.br)

# Regressão linear

![cover](https://miro.medium.com/max/1400/1*-hY-dDt5VlCDWeX7nmQfGw.png)

Neste módulo, falaremos de um dos primeiros e mais simples modelos de predição, a regressão linear. Apesar da sua simplicidade, a regressão linear possui grande poder preditivo e ainda é a ferramenta padrão para muitas áreas como, por exemplo, Econometria. Além disso, esses modelos trazem alguns detalhes teóricos que são facilmente esquecidos ou ignorados, mas que devem ter a devida atenção.

## Importação das bibliotecas

In [0]:
import functools
from math import sqrt

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import scipy.stats as sct
import seaborn as sns
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [0]:
# Algumas configurações para o matplotlib.
%matplotlib inline

from IPython.core.pylabtools import figsize


figsize(12, 12)

sns.set()

In [0]:
np.random.seed(1000)

## _Trade-off viés-variância_

Antes de falarmos sobre regressão, vale a pena introduzir o conceito do _trade-off_ viés-variância (_bias-variance trade-off_). O entendimento do _trade-off_ ajuda a compreender algumas decisões que podemos tomar ao escolher e comparar modelos de regressão.

Para avaliar um modelo, podemos observar o quanto esse modelo erra ao fazer predições, de acordo com alguma funções de erro. É possível mostrar que o erro cometido por um modelo sempre pode ser decomposto em três partes: o viés, a variância e um erro irredutível:

$$\mathbb{E}[(f(x) - \hat{f}(x))^{2}] = \underbrace{\text{Bias}(\hat{f}(x))^{2}}_{\text{viés}} + \underbrace{\text{Var}[\hat{f}(x)]}_{\text{variância}} + \underbrace{\varepsilon^{2}}_{\text{erro irredutível}}$$

onde

$$\text{Bias}(\hat{f}(x)) = \mathbb{E}[\hat{f}(x)] - f(x)$$

e

$$\text{Var}[\hat{f}(x)] = \mathbb{E}[\hat{f}(x)^{2}] - \mathbb{E}[\hat{f}(x)]^{2}$$

O viés se deve à escolha do modelo, que pode ter sido muito simples para o problema em mãos. Por exemplo, se temos um problema altamente não linear e escolhemos um modelos linear simples, introduziremos o nosso viés na escolha do modelo, resultando em predições fracas e uma baixa performance. Geralmente, o viés é relacionado _underfitting_ dos dados.

A variância, por outro lado, está relacionada à complexidade do modelo. Isso se traduz em pequenas variações dos dados resultando em grandes variações nas predições, o que não deveria acontecer. Um modelo com alta variância, por exemplo um modelo de regressão com excessivas _features_ polinomiais, geralmente apresenta _overfitting_ dos dados.

O erro irredutível é parte inerente do modelo e que não pode ser reduzido. Para reduzirmos o erro do modelo, devemos nos focar então em reduzir o viés ou a variância.

Acontece que não podemos reduzir ambos simultaneamente: ao reduzirmos o viés, aumentamos a variância e vice-versa. Por isso trata-se de um _trade-off_. Devemos escolher reduzir o viés ou a variância (o que for possível com o modelo e o que levar aos melhores resultados).

![mse-train-test](http://www.thefactmachine.com/wp-content/uploads/2015/04/14-Ridge-TestTrainingGraph.jpg)

Note na figura acima como o MSE de treinamento é sempre descendente, enquanto o de teste tem a forma de um 'U'. Durante o treino, o modelo tenta se ajustar ao máximo ao conjunto de dados apresentado a ele (o conjunto de treinamento), levando a um "superaprendizado" desses dados. Chamamos isso de _overfitting_: o modelo não aprende a generalizar, e sim a decorar os dados apresentados durante o treinamento. Por outro lado, se o modelo for muito mais simples do que a realidade que se tenta modelar, observaremos _underfitting_: o modelo não consegue capturar as variações legítimas dos dados e, portanto, performa de forma não satisfatória.

![overfitting-underfitting](https://miro.medium.com/max/1400/1*9hPX9pAO3jqLrzt0IE3JzA.png)

O que estamos interessados é no erro cometido durante o teste (ou validação) do modelo, quando novos dados (não utilizados durante o treinamento) são apresentados ao modelo e sua performance é então avaliada. Quando o modelo tem pouca flexibilidade (alto viés e baixa variância), o modelo erra muito durante o teste, tendo alto erro e baixo desempenho. De acordo em que a flexibilidade do modelo aumenta (baixo viés e alta variância) o modelo começa a acertar mais (cometer menos erro) durante um tempo, e logo depois começa a errar mais novamente. O que queremos é exatamente o ponto intermediário onde ele erra o mínimo durante o teste. Esse é o ponto ótimo do modelo e os hiperparâmetros utilizados pelo modelo nesse momento são ótimos.

## Regressão linear

Agora vamos analisar o modelo de regressão linear. Trata-se de um modelo antigo, com seus primeiros estudos feitos por Legendre e Gauss, mas que só teve a nomeação de _regressão_ dada por Sir Francis Galton em seus estudos sobre alturas de pais e filhos.

Duas observações muito importantes antes de prosseguirmos:

1. A regressão linear possui uma série de suposições estatísticas e propriedades que são importantes e tentaremos apresentar aqui.
2. Apesar do nome, a regressão linear também lida com relações não lineares, basta introduzir variáveis polinomiais por exemplo.

O problema que tentamos resolver aqui é um problema de regressão: dadas variáveis independentes, desejamos prever uma variável numérica dependente.

Por exemplo, a partir de dados de sensores (temperatura, pressão, umidade etc), podemos estar interessados em predizer a resistência de um certo material. Os diversos nomes atribuídos a ambas variáveis dependentes e independentes no contexto da regressão são mostrados abaixo:

![terminology](https://drive.google.com/uc?export=download&id=1-0KctX0PRWhvHSj_bBbw4EuzwDs0sTJ3)

Em um problema de regressão, as variáveis independentes podem ser numéricas ou categóricas, enquanto a variável explicada é sempre numérica.

### Modelo teórico

Vamos começar com o caso mais simples de regressão linear, onde temos apenas uma variável independente. Chamaremos essa variável de $X$ e a variável dependente de $Y$. Nesse cenário, podemos dizer que o valor esperado de $Y$ para um valor $X_{i}$ de $X$ tem a forma

$$\mathbb{E}[Y | X_{i}] = f(X_{i})$$

Note que $f(X)$ é a "verdade", uma relação teórica com parâmetros desconhecidos.

Podemos imaginar que essa relação é estreitamente linear e dizer que

$$f(X_{i}) = \beta_{1} + \beta_{2} X_{i}$$

Chamamos a função $f(X_{i})$ de função de regressão populacional, FRP, e ela geralmente é desconhecida, assim como todo parâmetro populacional. Em outras palavras, não conhecemos os parâmetros $\beta_{0}$ e $\beta_{1}$ que descrevem exatamente a relação entre $X$ e $Y$.

![model](https://www.jmp.com/en_us/statistics-knowledge-portal/what-is-regression/simple-linear-regression-model/_jcr_content/par/styledcontainer_2069/par/lightbox_55ba/lightboxImage.img.gif/1562878184943.gif)

Para um dado valor $X_{i}$ de $X$, observamos um determinado valor $Y_{i}$ de $Y$. Para não trabalharmos mais com $\mathbb{E}[Y | X_{i}]$ e sim com $Y_{i}$, precisamos introduzir o termo de erro:

$$Y_{i} = \beta_{1} + \beta_{2} X_{i} + u_{i}$$

Como não conhecemos a FRP, vamos estimá-la a partir de uma função de regressão amostral, FRA. Podemos escrever a FRA como

$$Y_{i} = \hat{f}(X_{i}) = \hat{\beta}_{1} + \hat{\beta}_{2} X_{i} + \hat{u}_{i}$$

A relação entre a FRP (PRF), a FRA (SRF) e suas componentes é ilustrada abaixo:

![relation](https://drive.google.com/uc?export=download&id=1I1gLeak5Vz8OVSd_hAoC1efo-XCDdc-x)

Essa base teórica nos serve para entendermos de forma mais sólida o problema estatístico por baixo dos panos. Toda análise feita com apenas uma variável independente aqui se estende diretamente ao problema com múltiplas variáveis.

### Modelo prático

Entendido o problema teórico, vamos partir para uma versão mais simples e prática do problema, já com múltiplas variáveis. O que queremos é estimar uma variável $y$ a partir de dados de $p$ variáveis $x$ através de um modelo de regressão. Isso pode ser escrito como

$$\hat{y}_{i} = \hat{\beta}_{0} + \hat{\beta}_{1} x_{i1} + \hat{\beta}_{2} x_{i2} + \cdots + \hat{\beta}_{p} x_{ip}$$

onde $i$ denota a $i$-ésima observação do conjunto de treino.

Como $\hat{y}$ é uma aproximação para o verdadeiro $y$, então, para cada observação $i$ da amostra, existe um erro/resíduo associado $e_{i}$ entre o valor observado e a predição:

$$e_{i} = y_{i} - \hat{y}_{i}$$

Na figura abaixo, a curva em azul denota $\hat{y}$, enquanto os pontos vermelhos são os valores $y_{i}$ da amostra. As barras pretas entre $y_{i}$ e $\hat{y}_{i}$ denotam os resíduos, ou erros de predição, $e_{i}$.

![residual](https://uc-r.github.io/public/images/analytics/regression/sq.errors-1.png)

A soma de todos $e_{i}$ ($i = 1, 2, \cdots, n$) é chamado de soma quadrática dos resíduos, RSS (_Residual Sum of Squares_):

$$\text{RSS} = \sum_{1 \leq i \leq n} e_{i}^{2} = \sum_{1 \leq i \leq n} (y_{i} - \hat{y}_{i})^{2}$$

É uma variação dela que queremos minimizar durante o treino, o MSE (_Mean Squared Error_):

$$\text{MSE} = \frac{1}{n} \sum_{1 \leq i \leq n} e_{i}^{2} = \frac{1}{n} \sum_{1 \leq i \leq n} (y_{i} - \hat{y}_{i})^{2}$$

Em outras palavras, o problema que queremos resolver é minimizar a seguinte função:

$$\underset{\hat{\beta}_{0}, \hat{\beta}_{2}, \cdots, \hat{\beta}_{p}}{\text{minimizar}} \text{MSE} = \frac{1}{n} \sum_{1 \leq i \leq n} (y_{i} - \hat{\beta}_{0} - \sum_{1 \leq j \leq p} \hat{\beta}_{j}x_{ij})^{2}$$

Apesar disso, para avaliar o nosso modelo após treinado, utilizamos o RMSE (_Root Mean Square Error_):

$$\text{RMSE} = \sqrt{\frac{1}{n} \sum_{1 \leq i \leq n} e_{i}^{2} = \sum_{1 \leq i \leq n} (y_{i} - \hat{y}_{i})^{2}}$$

O motivo para isso é que, devido suas propriedades analíticas (derivação etc), é bem mais fácil estimar os valores de $\beta$ utilizando o MSE do que utilizando o RMSE.

O RMSE é o erro-padrão dos erros. Ele é uma medida de quanto o nosso modelo erra ao predizer valores que não foram usadas durante o treinamento.

Um jeito de encontrar os valores de $\hat{\beta}_{0}, \hat{\beta}_{1}, \cdots, \hat{\beta}_{p}$, ou seja, de treinar o nosso modelo é utilizando a equação normal:

$$\hat{\beta} = (X^{T}X)^{-1} X^{T} y$$

É muito comum utilizar a matriz pseudoinversa de Moore-Penrose, $X^{+}$, dada sua facilidade de computação e maior robustez em casos excepcionais:

$$\hat{\beta} = X^{+} y$$


Para ver isso na prática, vamos utilizar o _data set_ famoso de casas de Boston (sim, ele mesmo):

In [0]:
boston_dataset = load_boston()

boston_features = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston_target = pd.DataFrame(boston_dataset.target, columns=["Price"])

boston = pd.concat([boston_features, boston_target], axis=1)

boston.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [0]:
print(boston_dataset.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

Vamos começar separando o nosso conjunto de dados em dois grupos: treinamento (80%) e teste (20%).

In [0]:
boston_features_train, boston_features_test, boston_target_train, boston_target_test = train_test_split(boston_features,
                                                                                                        boston_target,
                                                                                                        test_size=0.2,
                                                                                                        random_state=42)

Vamos treinar o modelo com todos os dados do conjunto primeiro:

In [0]:
linear_regression = LinearRegression()

linear_regression.fit(boston_features_train, boston_target_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Utilizaremos os dados do conjunto de teste para fazer predições. Note que nenhum desses dados de teste foram utilizados durante o treinamento do modelo:

In [0]:
predicted = linear_regression.predict(boston_features_test)

predicted[:5]

array([[28.99672362],
       [36.02556534],
       [14.81694405],
       [25.03197915],
       [18.76987992]])

Os verdadeiros valores de preço para os dados previstos acima são mostrados abaixo:

In [0]:
boston_target_test.values[:5]

array([[23.6],
       [32.4],
       [13.6],
       [22.8],
       [16.1]])

Podemos verificar os valores do parâmetros estimados, excluindo o intercepto, através do atributo `coef_` disponível no modelo após o treinamento (i.e., após a chamada da função `fit()`):

In [0]:
linear_regression.coef_.round(2)

array([[-1.10e-01,  3.00e-02,  4.00e-02,  2.78e+00, -1.72e+01,  4.44e+00,
        -1.00e-02, -1.45e+00,  2.60e-01, -1.00e-02, -9.20e-01,  1.00e-02,
        -5.10e-01]])

O valor do intercepto pode ser obtido através do atributo `intercept_`:

In [0]:
linear_regression.intercept_.round(2)

array([30.25])

Para mostrar a matemática por trás do treinamento desses modelos, vamos criar um `np.ndarray` com as variáveis e os valores de preço:

In [0]:
boston_complete_train = np.c_[np.ones(boston_features_train.shape[0]), boston_features_train.values]

E utilizar as operações matriciais e vetoriais disponíveis no NumPy para calcular a equação normal:

$$\hat{\beta} = (X^{T}X)^{-1} X^{T} y$$



In [0]:
np.linalg.inv(boston_complete_train.T.dot(boston_complete_train)).dot(boston_complete_train.T).dot(boston_target_train.values).round(2)

array([[ 3.025e+01],
       [-1.100e-01],
       [ 3.000e-02],
       [ 4.000e-02],
       [ 2.780e+00],
       [-1.720e+01],
       [ 4.440e+00],
       [-1.000e-02],
       [-1.450e+00],
       [ 2.600e-01],
       [-1.000e-02],
       [-9.200e-01],
       [ 1.000e-02],
       [-5.100e-01]])

Repare como os valores dos parâmetros estimados acima são iguais aos valores retornados pelo modelo em `linear_regression`.

Uma outra forma de calcular os mesmos parâmetros é utilizar a função `lstsq()` também do NumPy. Ela calcula exatamente o produto da matriz pseudoinversa de Moore-Penrose pela variável dependente:

$$\hat{\beta} = X^{+} y$$

In [0]:
coeff_svd, _res, _rank, _s = np.linalg.lstsq(boston_complete_train, boston_target_train.values, rcond=1e-6)

coeff_svd.round(2)

array([[ 3.025e+01],
       [-1.100e-01],
       [ 3.000e-02],
       [ 4.000e-02],
       [ 2.780e+00],
       [-1.720e+01],
       [ 4.440e+00],
       [-1.000e-02],
       [-1.450e+00],
       [ 2.600e-01],
       [-1.000e-02],
       [-9.200e-01],
       [ 1.000e-02],
       [-5.100e-01]])

Novamente, chegamos aos mesmos valores estimados.

Para finalizar, poderíamos ter utilizado a função `pinv()` que calcula a matriz pseudoinversa de Moore-Penrose (que depois podemos ainda multiplicar pela variável independente):

In [0]:
np.linalg.pinv(boston_complete_train).dot(boston_target_train.values).round(2)

array([[ 3.025e+01],
       [-1.100e-01],
       [ 3.000e-02],
       [ 4.000e-02],
       [ 2.780e+00],
       [-1.720e+01],
       [ 4.440e+00],
       [-1.000e-02],
       [-1.450e+00],
       [ 2.600e-01],
       [-1.000e-02],
       [-9.200e-01],
       [ 1.000e-02],
       [-5.100e-01]])

A partir do vetor de parâmetroes estimados, podemos prever o valor do preço de uma casa a partir das variáveis:

In [0]:
first_house = boston_complete_train[0, :]

first_house_predicted = first_house.dot(coeff_svd)

first_house_predicted

array([10.96952405])

## _Ridge_

A regressão _ridge_ é uma forma de regularizar a regressão linear. O objetivo é aprender um modelo de regressão ao mesmo tempo em que mantém reduzidas as magnitudes dos parâmetros aprendidos.

Para isso, adicionamos um termo de regularização à função de custo:

$$J^{\text{ridge}}(\hat{\beta}) = \frac{1}{n} \sum_{1 \leq i \leq n} (y_{i} - \hat{\beta}_{0} - \sum_{1 \leq j \leq p} \hat{\beta}_{j}x_{ij})^{2} + \underbrace{\lambda \sum_{1 \leq j \leq p} \hat{\beta}_{j}^{2}}_{\text{termo de regularização}}$$

Note como no termo de regularização, consideramos apenas os parâmetros $\hat{\beta}$ de índices 1 a $p$. Isso porque não queremos regularizar o intercepto ($\hat{\beta}_{0}$).

O termo $\lambda$ regula o quanto queremos regularizar o modelo. Para $\lambda = 0$, a regressão _ridge_ se torna uma regressão linear sem regularização. Conforme aumentamos o valor de $\lambda$, mais regularizado se torna o modelo, ou seja, menores serão os valores de $\beta$ estimados.

O parâmetro $\lambda$ é um hiperparâmetro do modelo e deve ser ajustado com métodos como _cross-validation_. A escolha do $\lambda$ correto é vital para o bom desempenho do _ridge_.

Apesar de tornar os parâmetros estimados valores muito pequenos, a regressão _ridge_ não os zera por completo. Em outras palavras, não somos capazes de fazer seleção de variáveis naturalmente com a regressão _ridge_.

O poder da regressão _ridge_ vem justamente do _trade-off bias-variance_. A regressão sem regularização ($\lambda = 0$) possui grande variância e pouco viés. À medida em que aumentamos $\lambda$, a variância diminui a uma taxa maior do que o viés aumenta, reduzindo assim o MSE total. A partir de certo ponto, o viés começa a aumentar mais rápido do que a variância aumenta, tornando a aumenta o MSE. Assim, existe um valor intermediário onde o MSE da regressão _ridge_ encontra seu mínimo e que é menor que o MSE da regressão linear sem regularização.

Importante notar que realizamos a regularização somente durante o treino. Em todas etapas seguintes (teste, validação, e predição), devemos avaliar a performance do modelo __sem__ regularização.

Outro ponto importante é que devemos escalar os dados antes de treinar a regressão _ridge_ utilizando o `StandardScaler`.

A regressão _ridge_ também é chamada de regressão $\ell_{2}$, pois utiliza a norma-$\ell_{2}$:

$$\|\hat{\beta}\|_{2} = \sqrt{\sum_{j} \hat{\beta}_{j}^{2}}$$

Vamos utilizar a regressão _ridge_ abaixo:

Começamos escalando os dados:

In [0]:
scaler = StandardScaler()

boston_features_train_scaled = scaler.fit_transform(boston_features_train)

boston_features_train_scaled[:3]

array([[ 1.28770177, -0.50032012,  1.03323679, -0.27808871,  0.48925206,
        -1.42806858,  1.02801516, -0.80217296,  1.70689143,  1.57843444,
         0.84534281, -0.07433689,  1.75350503],
       [-0.33638447, -0.50032012, -0.41315956, -0.27808871, -0.15723342,
        -0.68008655, -0.43119908,  0.32434893, -0.62435988, -0.58464788,
         1.20474139,  0.4301838 , -0.5614742 ],
       [-0.40325332,  1.01327135, -0.71521823, -0.27808871, -1.00872286,
        -0.40206304, -1.6185989 ,  1.3306972 , -0.97404758, -0.60272378,
        -0.63717631,  0.06529747, -0.65159505]])

In [0]:
ridge_regression = Ridge(alpha=1, solver="cholesky")

ridge_regression.fit(boston_features_train_scaled, boston_target_train)

ridge_regression.intercept_, ridge_regression.coef_

(array([22.79653465]),
 array([[-0.99218679,  0.6777488 ,  0.2522143 ,  0.72248078, -1.99083465,
          3.15157218, -0.17726162, -3.04502895,  2.17324941, -1.69555879,
         -2.02783351,  1.127197  , -3.59897667]]))

Note como os valores dos parâmetros estimados são bem próximos de zero (exceto o intercepto), mas nenhum exatamente zero.

![mse-lambda](https://drive.google.com/uc?export=download&id=196MmaQswPNPhBt46zttNJbffFny3bBGH)

A figura acima mostra a relação entre o erro (MSE) e o valor de $\lambda$. A curva preta representa a variância, a curva preta representa o viés, a linha tracejada representa o erro irredutível, e a curva rosa representa o MSE, ou seja, a soma da variância com o quadrado do viés e o erro irredutível.

Note como, até certo ponto, a variância diminui mais rápido do que o viés aumenta. Depois o viés aumenta muito mais rapidamente do que a variância consegue diminuir. Em algum ponto intermediário, encontramos o valor ótimo de $\lambda$, quando o erro total, MSE, é mínimo.

## LASSO

LASSO (_Least Absolute Shrinkage and Selection Operator_) é outra forma de regularização de regressão. Em vez de utilizar a norma-$\ell_{2}$, o LASSO utiliza a norma-$\ell_{1}$:

$$\|\hat{\beta}\|_{1} = \sqrt{\sum_{j} \| \hat{\beta}_{j} \|}$$

A função de custo com o novo termo de regularização adicionado se torna:

$$J^{\text{LASSO}}(\hat{\beta}) = \frac{1}{n} \sum_{1 \leq i \leq n} (y_{i} - \hat{\beta}_{0} - \sum_{1 \leq j \leq p} \hat{\beta}_{j}x_{ij})^{2} + \underbrace{\lambda \sum_{1 \leq j \leq p} \| \hat{\beta}_{j} \|}_{\text{termo de regularização}}$$

Essa pequena diferença entre as funções de custo do _ridge_ e do LASSO é o suficiente para termos um comportamente bastante diferente. Ao contrário do _ridge_, o LASSO é capaz de zerar completamente parâmetros de variáveis com pouca relevância para o modelo, gerando um modelo esparso (algumas das variáveis têm parâmetros nulos). Essa é uma forma de seleção de variáveis embutida no próprio modelo.

As mesmas observações do _ridge_ valem para o LASSO:

* Devemos escalar as variáveis antes de treinar o modelo do LASSO.
* Apenas utilizamos a função de custo com o termo de regularização adicionado durante o treino. Em etapas consecutivas (teste, validação, predição etc) devemos avaliar o desempenho do modelo com a função de custo simples, __sem__ regularização.

In [0]:
lasso_regression = Lasso(alpha=0.5)

lasso_regression.fit(boston_features_train_scaled, boston_target_train)

lasso_regression.intercept_, lasso_regression.coef_

(array([22.79653465]),
 array([-0.30737992,  0.        , -0.        ,  0.40396576, -0.        ,
         3.32512039, -0.        , -0.3333949 , -0.        , -0.        ,
        -1.46416614,  0.80194371, -3.53546088]))

In [0]:
boston_dataset.feature_names

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

Note como alguns parâmetros foram estimados como exatamente zero, removendo totalmente a respectiva variável do modelo final. As variáveis "sobreviventes" (não nulas) são as consideradas relevantes para a predição.

## Validação de regressão

Existe um conjunto de métricas para validarmos o desempenho dos modelos de regressão. Essas métricas podem ser utilizadas para comparar modelos e também para ajustar o valor de hiperparâmetros como o fator de regularização $\lambda$.

Vamos falar de alguns deles:

### $R^{2}$

O $R^{2}$, ou coeficiente de determinação, é uma métrica clássica para validação de regressão. Ele é definido como

$$R^{2} = 1 - \frac{\sum_{1 \leq i  \leq n} (y_{i} - \hat{y}_{i})^{2}}{\sum_{1 \leq i  \leq n} (y_{i} - \bar{y})^{2}}$$

Isso nada mais é que a razão da variância explicada pelo modelo pela variância total dos dados.

O $R^{2}$ é um valor que vai de 0 a 1. Quanto mais próximo de zero, menos o modelo explica a variância total, ou seja, menor é o desempenho do modelo. Quanto mais próximo de um, maior é a parcela da variância explicada pelo modelo, ou seja, melhor o modelo.

Importante notar que o $R^{2}$ não é, na verdade, o quadrado de alguma medida $R$, e portanto pode ser negativo. Isso pode acontecer, por exemplo, se o seu modelo não inclui o intercepto (e o modelo performa pior que o esperado). Se você estiver presenciando um $R^{2}$ negativo, verifique se o intercepto está sendo incluído no modelo.

O $R^{2}$ tem dois problemas: quanto mais variáveis há no modelo, maior o $R^{2}$ naturalmente. Um jeito de inflar artificialmente o desempenho do modelo é adicionando variáveis ao modelo, mesmo que elas não sejam de fato muito relevantes.

O segundo problema é decorrente do primeiro: não conseguimos comparar modelos com números diferentes de variáveis de forma justa. Não temos como saber se um possível $R^{2}$ maior no modelo com mais variáveis é devido a um melhor desempenho autêntico ou somente ao seu número maior de variáveis.

Para resolver isso, existe o $R^{2}$ ajustado:

$$R^{2}_{\text{ajustado}} = 1 - \frac{n-1}{n - (k + 1)} (1 - R^{2})$$

onde $n$ é o número de observações (tamanho da amostra) e $k + 1$ é o número de parâmetros sendo estimados, incluindo o intercepto.

In [0]:
r2_score(boston_target_test, predicted)

0.6687594935356307

### _Mean Squared Error_

Uma outra forma de medir o desempenho de um modelo é através do já visto MSE:

$$\text{MSE} = \frac{1}{n} \sum_{1 \leq i \leq n} (y_{i} - \hat{y}_{i})^{2}$$

Esse valor, ao contrário do $R^{2}$, é sempre estritamente não negativo (sendo zero no caso de um ajuste perfeito aos dados). Quanto mais próximo de zero, melhor o desempenho do modelo, pois as diferenças entre as predições e os reais valores é menor.

Esse valor sozinho não passa muitas informações, e portanto o MSE é melhor aproveitado ao comparar modelos. Por exemplo, podemos escolher o valor de $\lambda$ em uma regressão _ridge_ ou LASSO que leve ao menor MSE.

In [0]:
mean_squared_error(boston_target_test, predicted)

24.291119474973616

### _Median Absolute Error_

Outra medida interessante é o MedAE, a mediana dos erros de predição. Essa métrica é robusta à presença de _outliers_, pois considera os quantis dos resultados ao invés da média, que é muito mais sensível à variações extremas.

O MedAE é computado como:

$$\text{MedAE} = \text{mediana}\{y_{1} - \hat{y}_{1}, y_{2} - \hat{y}_{2}, \cdots, y_{n} - \hat{y}_{n}\}$$

Esse valor assume qualquer valor real e, como toda medida de erro, quanto menor, melhor o modelo. Assim como o MSE, esse valor por si só não diz muita coisa, e é melhor utilizado ao comparar modelos.

In [0]:
median_absolute_error(boston_target_test, predicted)

2.3243319064124535

## Referências

* [Gentle Introduction to the Bias-Variance Trade-Off in Machine Learning](https://machinelearningmastery.com/gentle-introduction-to-the-bias-variance-trade-off-in-machine-learning/)

* [Understanding the Bias-Variance Tradeoff](http://scott.fortmann-roe.com/docs/BiasVariance.html)

* [Introduction to Machine Learning Algorithms: Linear Regression](https://towardsdatascience.com/introduction-to-machine-learning-algorithms-linear-regression-14c4e325882a)

* [7 Classical Assumptions of Ordinary Least Squares (OLS) Linear Regression](https://statisticsbyjim.com/regression/ols-linear-regression-assumptions/)

* [The Gauss-Markov Theorem and BLUE OLS Coefficient Estimates](https://statisticsbyjim.com/regression/gauss-markov-theorem-ols-blue/)

* [Tikhonov regularization](https://en.wikipedia.org/wiki/Tikhonov_regularization)

* [Ridge Regression for Better Usage](https://towardsdatascience.com/ridge-regression-for-better-usage-2f19b3a202db)

* [Lasso](https://en.wikipedia.org/wiki/Lasso_(statistics))

* [Understanding Regression Error Metrics in Python](https://www.dataquest.io/blog/understanding-regression-error-metrics/)

* [Understand Regression Performance Metrics](https://becominghuman.ai/understand-regression-performance-metrics-bdb0e7fcc1b3)